# Feature engineering
This notebook will serve as a preliminar feature engineering conceptual approach to a training-ready dataset. There are similar types of features that will be treated with the same transformations.

In [127]:
import pandas as pd

In [128]:
stocks = pd.read_csv("../data/raw/faang_stock_prices.csv")

## Date
The Date field will be broken into different sections to make the features useful to the model:
- Year
- Quarter
- Month
- Day
- Day of week

First step is converting the original string data type to a datetime object and then do the splitting.

In [129]:
stocks["Date"] = pd.to_datetime(stocks["Date"])

And then use the df methods to get the features.

In [130]:
stocks["Year"] = stocks["Date"].dt.year
stocks["Quarter"] = stocks["Date"].dt.quarter
stocks["Month"] = stocks["Date"].dt.month
stocks["Day"] = stocks["Date"].dt.day
stocks["Day_of_week"] = stocks["Date"].dt.day_of_week

stocks.drop(["Date"], axis=1, inplace=True)

In [131]:
stocks.sample(5)[["Year", "Quarter", "Month", "Day", "Day_of_week"]]

,Year,Quarter,Month,Day,Day_of_week
10556,2023,1,2,16,3
10967,2023,2,5,25,3
9829,2022,3,8,24,2
9460,2022,2,5,25,2
809,2016,3,9,1,3


## Ticker
One hot encoding for the companies tickers.

In [132]:
one_hot = pd.get_dummies(stocks["Ticker"], prefix="Ticker")
stocks = stocks.drop("Ticker", axis=1).join(one_hot)

## Price-Action features
These features describe the "shape" of the current day trading prices, but relatives to its final price.

The *volume* feature is transformed by applying a log function (to compensate for massive volumes in some days).

In [133]:
import numpy as np
stocks['Rel_Open'] = (stocks['Open'] - stocks['Close']) / stocks['Close']
stocks['Rel_High'] = (stocks['High'] - stocks['Close']) / stocks['Close']
stocks['Rel_Low'] = (stocks['Low'] - stocks['Close']) / stocks['Close']
stocks['Log_Volume'] = np.log(stocks['Volume']) / 10

stocks.drop(['Open', 'High', 'Low', 'Volume'], axis=1, inplace=True)


## Rolling means: SMA and EMA
For this rolling means, we will use the close price as the reference for calculating the means as ratios. So the formula will be:
$$
\text{SMA}_r = \frac{\text{Close}_i}{\text{SMA}_i}
$$
And the EMA will be:
$$
\text{EMA}_r = \frac{\text{Close}_i}{\text{EMA}_i}
$$

In [134]:
for col in ['SMA_7', 'SMA_21', 'EMA_12', 'EMA_26']:
    stocks[f'R{col}'] = stocks['Close'] / stocks[col]
    stocks.drop(col, axis=1, inplace=True)

## Momentum and volatility
These are often already "bounded" (within a set range), so they are simply scaled them to be between 0 and 1.

- RSI: 0 to 100 -> 0 to 1
- MACD and Signal MACD -> Realtive to stock close price
- Bollinger (upper and lower) -> Bollinger_P (relative postion of the close price in the Bollinger bands)

*Daily return* and *Volatility_7d* are keeped as they are.

In [135]:
stocks['SRSI'] = stocks['RSI_14'] / 100
stocks['RMACD'] = stocks['MACD'] / stocks['Close']
stocks['RSMACD'] = stocks['MACD_Signal'] / stocks['Close']
stocks['Bollinger_P'] = (stocks['Close'] - stocks['Bollinger_Lower']) / (stocks['Bollinger_Upper'] - stocks['Bollinger_Lower'])

stocks.drop(['RSI_14', 'MACD', 'MACD_Signal', 'Bollinger_Lower', 'Bollinger_Upper'], axis=1, inplace=True)

## Target: Next day return
The target will be converted to a percentage of the next day's closing price relative to the current day's closing price.

In [136]:
stocks["Next_Day_Return"] = (stocks["Next_Day_Close"] - stocks["Close"]) / stocks["Close"]

stocks.drop(["Daily_Return"], axis=1, inplace=True)

In [138]:
stocks.columns

Index(['Close', 'Volatility_7d', 'Next_Day_Close', 'Year', 'Quarter', 'Month',
       'Day', 'Day_of_week', 'Ticker_AAPL', 'Ticker_AMZN', 'Ticker_GOOGL',
       'Ticker_META', 'Ticker_MSFT', 'Ticker_NVDA', 'Rel_Open', 'Rel_High',
       'Rel_Low', 'Log_Volume', 'RSMA_7', 'RSMA_21', 'REMA_12', 'REMA_26',
       'SRSI', 'RMACD', 'RSMACD', 'Bollinger_P', 'Next_Day_Return'],
      dtype='str')